In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd

In [2]:
product_df = pd.read_csv("Product Wise ARR Report - Sheet5 (1).csv").drop(['Subscription Product ARR (converted)'], axis=1)
product_df['Product Name'] = product_df['Product Name'].str.replace('Training Essentials ', 'Training Essentials', regex=False)
product_df

,Account Name,Product Name,Quantity,Offered Price (converted)
0,BoostUp.ai,Training Essentials,6,10.93
1,AltiSales,Training Essentials,8,6.48
2,Paytronix,Training Essentials,8,5.01
3,APS Payroll,Training Essentials,10,6.00
4,ActZero,Training Essentials,10,15.83
...,...,...,...,...
1070,NuVasive,Virtual Role-Play (Missions + Quick Update),150,14.49
1071,Mendix EMEA,Virtual Role-Play (Missions + Quick Update),210,8.70
1072,Amplitude,Virtual Role-Play (Missions + Quick Update),475,3.99
1073,Alcon,Virtual Role-Play (Missions + Quick Update),1500,7.07


In [3]:
product_df = product_df.groupby(['Account Name', 'Product Name']).apply(lambda x: pd.Series({'Total Quantity': x['Quantity'].sum(),
    'Weighted Average Price': (x['Quantity'] * x['Offered Price (converted)']).sum() / x['Quantity'].sum() }))

product_df = product_df.reset_index()

## Calculating the Product Embedding using the formula = (Quantity * Weightage Average) / No_of_accounts
product_embed = product_df.groupby(['Product Name']).apply(lambda x: pd.Series({'Product Embed': (x['Total Quantity'] * x['Weighted Average Price']).sum() / x['Account Name'].count()})).reset_index()
product_df = product_df.drop(['Account Name'], axis=1)
product_df

,Product Name,Total Quantity,Weighted Average Price
0,Asset Hub,55.0,10.000000
1,Call AI,121.0,34.549587
2,Coaching,66.0,10.000000
3,Digital Sales Rooms,51.0,8.000000
4,Call AI,6.0,65.970000
...,...,...,...
780,Essentials Package,30.0,4.840000
781,Premier Package + Spaced Reinforcements,26.0,32.940000
782,Spaced Reinforcements,30.0,3.630000
783,Coaching,500.0,0.490000


In [4]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
# Adjusting the MinMaxScaler to normalize between 1 and len(product_embed)
n_rows = len(product_embed)

# Creating a new scaler with the desired range
scaler = MinMaxScaler(feature_range=(1, n_rows))

# Normalizing 'Product Weighted Average Price' column
product_embed['Product Embed'] = scaler.fit_transform(product_embed[['Product Embed']])
product_embed

,Product Name,Product Embed
0,Asset Hub,1.033883
1,Call AI,1.004078
2,Coaching,1.004422
3,Digital Sales Rooms,1.002308
4,Essentials Package,1.003180
5,Practice & Reinforcement,1.004279
6,Premier Package,1.001021
7,Premier Package + Spaced Reinforcements,1.016618
8,Professional Package,1.000000
9,Professional Package + Spaced Reinforcements,1.010350


In [6]:
product_df = product_df.merge(product_embed, on=['Product Name'])
product_df

,Product Name,Total Quantity,Weighted Average Price,Product Embed
0,Asset Hub,55.0,10.000000,1.033883
1,Asset Hub,32.0,19.315000,1.033883
2,Asset Hub,20.0,12.000000,1.033883
3,Asset Hub,30.0,13.666667,1.033883
4,Asset Hub,500.0,9.550000,1.033883
...,...,...,...,...
780,Professional Package,105.0,16.530000,1.000000
781,Professional Package,60.0,15.300000,1.000000
782,Professional Package,30.0,16.430000,1.000000
783,Professional Package,10.0,23.450000,1.000000


In [7]:
product_df.groupby(['Product Name']).size().reset_index(name="count")

,Product Name,count
0,Asset Hub,98
1,Call AI,59
2,Coaching,81
3,Digital Sales Rooms,55
4,Essentials Package,19
5,Practice & Reinforcement,118
6,Premier Package,22
7,Premier Package + Spaced Reinforcements,90
8,Professional Package,11
9,Professional Package + Spaced Reinforcements,23


# Polynomial Regularisation

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, make_scorer

import matplotlib.pyplot as plt

from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, RandomizedSearchCV

from scipy.stats import uniform, randint
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import MinMaxScaler
import os

# L2 - Ridge Regularization

### Hyperparameter tuning considering the number of variables as well

# Generalized Code

In [9]:
import os
os.makedirs('Polynomial_images', exist_ok=True)

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from scipy.stats import uniform, expon

In [12]:
product_df.head(2)

,Product Name,Total Quantity,Weighted Average Price,Product Embed
0,Asset Hub,55.0,10.000,1.033883
1,Asset Hub,32.0,19.315,1.033883


In [ ]:
polynomial_degree_df = pd.DataFrame()
for grouped_value, grouped_df in product_df.groupby(['Product Name']):

    # Handling outliers for 'Total Quantity'
    Q1_quantity = grouped_df['Total Quantity'].quantile(0.25)
    Q3_quantity = grouped_df['Total Quantity'].quantile(0.75)
    IQR_quantity = Q3_quantity - Q1_quantity
    lower_bound_quantity = Q1_quantity - 1.5 * IQR_quantity
    upper_bound_quantity = Q3_quantity + 1.5 * IQR_quantity
    grouped_df['Total Quantity'] = np.where(grouped_df['Total Quantity'] < lower_bound_quantity, lower_bound_quantity, grouped_df['Total Quantity'])
    grouped_df['Total Quantity'] = np.where(grouped_df['Total Quantity'] > upper_bound_quantity, upper_bound_quantity, grouped_df['Total Quantity'])

    # Handling outliers for 'Weighted Average Price'
    Q1_price = grouped_df['Weighted Average Price'].quantile(0.25)
    Q3_price = grouped_df['Weighted Average Price'].quantile(0.75)
    IQR_price = Q3_price - Q1_price
    lower_bound_price = Q1_price - 1.5 * IQR_price
    upper_bound_price = Q3_price + 1.5 * IQR_price
    grouped_df['Weighted Average Price'] = np.where(grouped_df['Weighted Average Price'] < lower_bound_price, lower_bound_price, grouped_df['Weighted Average Price'])
    grouped_df['Weighted Average Price'] = np.where(grouped_df['Weighted Average Price'] > upper_bound_price, upper_bound_price, grouped_df['Weighted Average Price']) 
    
    X = grouped_df['Total Quantity'].values.reshape(-1, 1)
    y = grouped_df['Weighted Average Price']

    # Splitting the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

    for degree in range(2,6):
        poly_features = PolynomialFeatures(degree=degree)
        X_train_poly = poly_features.fit_transform(X_train)
        X_test_poly = poly_features.transform(X_test)

        # Setting up Ridge Regression model
        ridge_model = Ridge(random_state=0)

        # Defining the parameter distribution for alpha
        param_dist = {
            'alpha': uniform(0.0001, 10000)  # Uniform distribution for alpha values
        }

        # Setting up RandomizedSearchCV for hyperparameter tuning
        random_search = RandomizedSearchCV(ridge_model, param_distributions=param_dist, 
                                           n_iter=100, cv=5, random_state=0)

        # Running the random search to find the best hyperparameters
        random_search.fit(X_train_poly, y_train)

        # Training the model using the best parameters
        best_model = random_search.best_estimator_
        best_model.fit(X_train_poly, y_train)

        # Predicting
        y_pred = best_model.predict(X_test_poly)

        # Evaluating the model
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        n = len(y_test)  # Number of observations
        p = degree  # Number of predictors
        adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)

        print(f'Mean Squared Error: {mse}')
        print(f'R² Score: {r2}')
        print(f'Adjusted R² Score: {adjusted_r2}')
        print(f'Best Alpha: {random_search.best_params_["alpha"]}')

        # Plotting
        plt.scatter(X_train, y_train, color='green', label='Training Data')
        plt.scatter(X_test, y_test, color='red', label='Testing Data')
        plt.scatter(X_test, y_pred, color='blue', label='Predicted Values')

        plt.xlabel('Total Quantity')
        plt.ylabel('Weighted Average Price')
        plt.title(f'Ridge Regression Degree {degree}')    
        plt.legend()
        plot_file_path = f'Polynomial_images/{grouped_value}_plot_degree_{degree}.png'
        plt.savefig(plot_file_path)    
        plt.close()

        # Extracting coefficients and intercept
        coefficients = best_model.coef_
        intercept = best_model.intercept_

        # Constructing the polynomial equation as a string
        polynomial_terms = [f"{coeff:.3f} * X^{i}" for i, coeff in enumerate(coefficients) if coeff != 0]
        polynomial_equation = " + ".join(polynomial_terms)
        polynomial_equation = f"y = {intercept:.3f} + " + polynomial_equation

        print("Polynomial Regression Equation:")
        print(polynomial_equation)

        df = pd.DataFrame({
            "Product Name":[grouped_value],
            "Degree": [degree],
            "Mean Squared Error": [mse],
            "R² Score": [r2],
            "Adjusted R² Score": [adjusted_r2],
            "Best Alpha": [random_search.best_params_["alpha"]],
            "Polynomial Equation": [polynomial_equation],
            "Plot Image Path": [plot_file_path]
        })

        polynomial_degree_df = pd.concat([polynomial_degree_df, df], ignore_index=True)

In [ ]:
polynomial_degree_df

In [ ]:
# Grouping by 'Product Name' and finding the index of minimum 'Mean Squared Error' for each group
idx = polynomial_degree_df.groupby('Product Name')['Adjusted R² Score'].idxmax()
best_models_df = polynomial_degree_df.loc[idx]
best_models_df.reset_index(drop=True, inplace=True)
best_models_df

# Save the trained Models

In [ ]:
import os
import joblib
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline

os.makedirs('Models', exist_ok=True)

In [ ]:
for index, row in best_models_df.iterrows():
    product_name = row['Product Name']
    degree = row['Degree']
    alpha = row['Best Alpha']
    grouped_df = product_df[product_df['Product Name'] == product_name]

    X = grouped_df['Total Quantity'].values.reshape(-1, 1)
    y = grouped_df['Weighted Average Price']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

    model = make_pipeline(StandardScaler(), PolynomialFeatures(degree=degree), Ridge(alpha=alpha, random_state=0))
    model.fit(X_train, y_train)

    # Save the model
    filename = f'Models/model_{product_name}_degree_{degree}_alpha_{alpha:.4f}.joblib'
    joblib.dump(model, filename)

# Prediction Using the model

In [ ]:
import os
import joblib

def predict_price(product_name, total_quantity):
    models_dir = 'Models'

    model_file = None
    for file in os.listdir(models_dir):
        if product_name in file:
            model_file = file
            break

    if model_file is None:
        return f"No model found for product: {product_name}"

    # Load the model
    model_path = os.path.join(models_dir, model_file)
    model = joblib.load(model_path)

    # Make a prediction
    predicted_price = model.predict([[total_quantity]])

    return predicted_price[0]

In [ ]:
product_name = 'Training Essentials' 
total_quantity = 500
predicted_price = predict_price(product_name, total_quantity)
print(f"Predicted Price: {predicted_price}")

In [ ]:
product_df[(product_df['Product Name'] == product_name) & (product_df['Total Quantity'] == 500)]#.drop_duplicates(subset=['Product Name', 'Total Quantity']))

In [ ]:
product_df[product_df['Product Name'] == product_name]['Weighted Average Price'].describe()